In [1]:
from google_play_scraper import Sort, reviews
import pandas as pd
import time

import os
import sys

# Load other functions
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../scripts'))

from data_scraper import ScrapData
from sentiment_analysis import sentimentAnalysis
from Thematic_Analysis import ThematicAnalysis

C:\Users\Specter\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Specter\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
# Initialize scraper
scraper = ScrapData()

In [6]:
# App IDs
apps = {
    #"CBE": "com.combanketh.mobilebanking",
    #"BOA": "com.boa.boaMobileBanking",
    "Dashen": "com.cr2.amolelight"
}
all_data = []


In [5]:
# Fetch reviews for each app
for app_name, app_id in apps.items():
    print(f"\nFetching reviews for {app_name}...")
    df = scraper.fetch_reviews(app_id, app_name, n_reviews=400)
    print(f"{app_name} - {len(df)} reviews fetched.")
    all_data.append(df)

# Combine all into one DataFrame
Banks_df = pd.concat(all_data, ignore_index=True)


Fetching reviews for Dashen...
Fetched 400 unique reviews for Dashen
Dashen - 400 reviews fetched.


In [6]:

Banks_df.to_csv("../Data/Dashen_reviews_raw.csv", index=False)
print("✅ All reviews saved to 'bank_reviews_raw.csv'")


✅ All reviews saved to 'bank_reviews_raw.csv'


In [7]:
Dashin = pd.read_csv('../Data/Dashen_reviews_raw.csv')
CBE_BOA = pd.read_csv('../Data/CBE&BOA_reviews_raw.csv')
Dashin

,reviewId,userName,content,score,at,app_name
0,2b25348c-1a20-482c-814e-648572cf6dd9,Hayat Abdulahi,best,5,2025-06-05 08:34:06,Dashen
1,21595904-0dbf-4eb3-bb27-98f67bd618ab,Tewodros Tsegayeed,nice application,4,2025-05-27 14:41:03,Dashen
2,b925631c-a63d-48d9-9852-68aed0898c12,gadisa worku,best,5,2025-05-22 23:11:40,Dashen
3,e0f868ee-be62-4ad5-8976-c5e3ee6f1cd9,Surafel Mershu,bast smart bank sistem dashen amole dashen bamk,5,2025-05-17 09:25:53,Dashen
4,b7d8df4f-45dc-44a1-ac3d-85d46177e7b8,Abdulkarim Abdala,best app,5,2025-05-16 16:30:27,Dashen
...,...,...,...,...,...,...
395,70951a6f-26ef-49de-8f9a-39384f78fccc,Anwar Kedir,Good and reliable service,5,2023-03-19 21:52:46,Dashen
396,15eb1d2a-87e5-4075-a961-dad92dd0bee7,Dashen Bank Deder Branch,"Best app I have ever seen woderfull, always on...",5,2023-03-19 14:06:24,Dashen
397,35ada4ca-0639-478b-b9a5-378ddb3769f7,Behailu Gashaw,Good apps,5,2023-03-19 13:51:25,Dashen
398,e331ee97-7303-40b9-a1ed-3fdd48498cb3,Ayanle Awcali,Good application for people needs,5,2023-03-18 23:28:12,Dashen


In [8]:
# Combine all reviews into a single DataFrame
Banks_df = pd.concat([CBE_BOA, Dashin])

In [9]:
Banks_df.shape

(1600, 6)

In [10]:
Banks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 399
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   reviewId  1600 non-null   object
 1   userName  1600 non-null   object
 2   content   1600 non-null   object
 3   score     1600 non-null   int64 
 4   at        1600 non-null   object
 5   app_name  1600 non-null   object
dtypes: int64(1), object(5)
memory usage: 87.5+ KB


In [11]:
Banks_df['app_name'].value_counts()

app_name
CBE       600
BOA       600
Dashen    400
Name: count, dtype: int64

In [12]:
Banks_df.duplicated().sum()

np.int64(0)

In [13]:
Banks_df.isna().sum()

reviewId    0
userName    0
content     0
score       0
at          0
app_name    0
dtype: int64

In [18]:
# Rename columns
Banks_df = Banks_df.rename(columns={
    'reviewId': 'reviewId',
    'userName': 'userName',
    'content': 'review',
    'score': 'rating',
    'at': 'date'
})

# Select and reorder necessary columns
Banks_df = Banks_df[['reviewId', 'userName', 'review', 'rating', 'date']]
Banks_df['bank'] = app_name
Banks_df['source'] = 'Google Play'


In [19]:
# Normalize dates
Banks_df['date'] = pd.to_datetime(Banks_df['date']).dt.date 

In [20]:
Banks_df = Banks_df[['reviewId', 'userName', 'review', 'rating', 'date', 'bank', 'source']]
Banks_df.to_csv("../Data/Preprocessed_bank_data.csv", index=False)
print(f"Saved preprocessed data to Preprocessed data")

Saved preprocessed data to Preprocessed data


In [2]:
merged_df = pd.read_csv('../Data/Preprocessed_bank_data.csv')
merged_df

,reviewId,userName,review,rating,date,bank,source
0,be2cb2ac-bbe0-4175-81c4-9f6c86afdaaa,Aim4 Beyond,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Dashen,Google Play
1,8efd71e9-59cd-41ce-8c5c-12052dee9ad0,zakir man,what is this app problem???,1,2025-06-05,Dashen,Google Play
2,b12d0383-9b27-4e49-a94d-277a43b15800,Yesuf Ahmed,the app is proactive and a good connections.,5,2025-06-05,Dashen,Google Play
3,dd9f9e37-177a-46df-b877-d0edaa9aed29,Yonas Mekonnen,I cannot send to cbebirr app. through this app.,3,2025-06-05,Dashen,Google Play
4,8e34703c-203c-4180-8b32-bfd0b3f0c871,Yibrah Yebo,good,4,2025-06-05,Dashen,Google Play
...,...,...,...,...,...,...,...
1595,70951a6f-26ef-49de-8f9a-39384f78fccc,Anwar Kedir,Good and reliable service,5,2023-03-19,Dashen,Google Play
1596,15eb1d2a-87e5-4075-a961-dad92dd0bee7,Dashen Bank Deder Branch,"Best app I have ever seen woderfull, always on...",5,2023-03-19,Dashen,Google Play
1597,35ada4ca-0639-478b-b9a5-378ddb3769f7,Behailu Gashaw,Good apps,5,2023-03-19,Dashen,Google Play
1598,e331ee97-7303-40b9-a1ed-3fdd48498cb3,Ayanle Awcali,Good application for people needs,5,2023-03-18,Dashen,Google Play


## Sentiment and Thematic Analysis

### VADER-based Sentiment

In [3]:
sentiment = sentimentAnalysis(merged_df)

In [5]:
sentiment.analyze_sentiment_vader()

,reviewId,userName,review,rating,date,bank,source,sentiment_vader
0,be2cb2ac-bbe0-4175-81c4-9f6c86afdaaa,Aim4 Beyond,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Dashen,Google Play,positive
1,8efd71e9-59cd-41ce-8c5c-12052dee9ad0,zakir man,what is this app problem???,1,2025-06-05,Dashen,Google Play,negative
2,b12d0383-9b27-4e49-a94d-277a43b15800,Yesuf Ahmed,the app is proactive and a good connections.,5,2025-06-05,Dashen,Google Play,positive
3,dd9f9e37-177a-46df-b877-d0edaa9aed29,Yonas Mekonnen,I cannot send to cbebirr app. through this app.,3,2025-06-05,Dashen,Google Play,neutral
4,8e34703c-203c-4180-8b32-bfd0b3f0c871,Yibrah Yebo,good,4,2025-06-05,Dashen,Google Play,positive
...,...,...,...,...,...,...,...,...
1595,70951a6f-26ef-49de-8f9a-39384f78fccc,Anwar Kedir,Good and reliable service,5,2023-03-19,Dashen,Google Play,positive
1596,15eb1d2a-87e5-4075-a961-dad92dd0bee7,Dashen Bank Deder Branch,"Best app I have ever seen woderfull, always on...",5,2023-03-19,Dashen,Google Play,positive
1597,35ada4ca-0639-478b-b9a5-378ddb3769f7,Behailu Gashaw,Good apps,5,2023-03-19,Dashen,Google Play,positive
1598,e331ee97-7303-40b9-a1ed-3fdd48498cb3,Ayanle Awcali,Good application for people needs,5,2023-03-18,Dashen,Google Play,positive


In [6]:
merged_df3 = merged_df.groupby('rating')['sentiment_vader'].value_counts()
merged_df3

rating  sentiment_vader
1       negative           166
        neutral            130
        positive            90
2       neutral             24
        negative            18
        positive            17
3       positive            49
        neutral             47
        negative            14
4       positive            76
        neutral             29
        negative             8
5       positive           686
        neutral            228
        negative            18
Name: count, dtype: int64

### BERT-based Sentiment (DistilBERT)

In [4]:
sentiment.analyze_sentiment_bert1()

Device set to use cpu


,reviewId,userName,review,rating,date,bank,source,sentiment_bert
0,be2cb2ac-bbe0-4175-81c4-9f6c86afdaaa,Aim4 Beyond,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Dashen,Google Play,negative
1,8efd71e9-59cd-41ce-8c5c-12052dee9ad0,zakir man,what is this app problem???,1,2025-06-05,Dashen,Google Play,negative
2,b12d0383-9b27-4e49-a94d-277a43b15800,Yesuf Ahmed,the app is proactive and a good connections.,5,2025-06-05,Dashen,Google Play,positive
3,dd9f9e37-177a-46df-b877-d0edaa9aed29,Yonas Mekonnen,I cannot send to cbebirr app. through this app.,3,2025-06-05,Dashen,Google Play,negative
4,8e34703c-203c-4180-8b32-bfd0b3f0c871,Yibrah Yebo,good,4,2025-06-05,Dashen,Google Play,positive
...,...,...,...,...,...,...,...,...
1595,70951a6f-26ef-49de-8f9a-39384f78fccc,Anwar Kedir,Good and reliable service,5,2023-03-19,Dashen,Google Play,positive
1596,15eb1d2a-87e5-4075-a961-dad92dd0bee7,Dashen Bank Deder Branch,"Best app I have ever seen woderfull, always on...",5,2023-03-19,Dashen,Google Play,positive
1597,35ada4ca-0639-478b-b9a5-378ddb3769f7,Behailu Gashaw,Good apps,5,2023-03-19,Dashen,Google Play,positive
1598,e331ee97-7303-40b9-a1ed-3fdd48498cb3,Ayanle Awcali,Good application for people needs,5,2023-03-18,Dashen,Google Play,positive


In [5]:
sentiment.sentiment_by_bank_and_rating()

,bank,rating,mean_sentiment
0,Dashen,1,-0.751295
1,Dashen,2,-0.694915
2,Dashen,3,-0.218182
3,Dashen,4,0.079646
4,Dashen,5,0.639485


In [6]:
sentiment.compare_star_rating_with_bert_sentiment()

Classification Report (excluding neutral ratings):

              precision    recall  f1-score   support

    negative       0.64      0.87      0.74       445
    positive       0.94      0.79      0.86      1045

    accuracy                           0.81      1490
   macro avg       0.79      0.83      0.80      1490
weighted avg       0.85      0.81      0.82      1490



,bank,review,rating,sentiment_rating_based,sentiment_bert
0,Dashen,"""Why don’t your ATMs support account-to-accoun...",4,positive,negative
1,Dashen,what is this app problem???,1,negative,negative
2,Dashen,the app is proactive and a good connections.,5,positive,positive
4,Dashen,good,4,positive,positive
5,Dashen,not functional,1,negative,negative
...,...,...,...,...,...
1595,Dashen,Good and reliable service,5,positive,positive
1596,Dashen,"Best app I have ever seen woderfull, always on...",5,positive,positive
1597,Dashen,Good apps,5,positive,positive
1598,Dashen,Good application for people needs,5,positive,positive


In [14]:
merged_d2f = merged_df.groupby('rating')['sentiment_bert'].value_counts()
merged_d2f

rating  sentiment_bert
1       negative          338
        positive           48
2       negative           50
        positive            9
3       negative           67
        positive           43
4       positive           61
        negative           52
5       positive          764
        negative          168
Name: count, dtype: int64

In [8]:
Theme = ThematicAnalysis(merged_df)

In [9]:
Theme.extract_themes()

,reviewId,userName,review,rating,date,bank,source,sentiment_vader,themes
0,be2cb2ac-bbe0-4175-81c4-9f6c86afdaaa,Aim4 Beyond,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,Dashen,Google Play,positive,[customer_service]
1,8efd71e9-59cd-41ce-8c5c-12052dee9ad0,zakir man,what is this app problem???,1,2025-06-05,Dashen,Google Play,negative,[other]
2,b12d0383-9b27-4e49-a94d-277a43b15800,Yesuf Ahmed,the app is proactive and a good connections.,5,2025-06-05,Dashen,Google Play,positive,[other]
3,dd9f9e37-177a-46df-b877-d0edaa9aed29,Yonas Mekonnen,I cannot send to cbebirr app. through this app.,3,2025-06-05,Dashen,Google Play,neutral,[other]
4,8e34703c-203c-4180-8b32-bfd0b3f0c871,Yibrah Yebo,good,4,2025-06-05,Dashen,Google Play,positive,[other]
...,...,...,...,...,...,...,...,...,...
1595,70951a6f-26ef-49de-8f9a-39384f78fccc,Anwar Kedir,Good and reliable service,5,2023-03-19,Dashen,Google Play,positive,[other]
1596,15eb1d2a-87e5-4075-a961-dad92dd0bee7,Dashen Bank Deder Branch,"Best app I have ever seen woderfull, always on...",5,2023-03-19,Dashen,Google Play,positive,[other]
1597,35ada4ca-0639-478b-b9a5-378ddb3769f7,Behailu Gashaw,Good apps,5,2023-03-19,Dashen,Google Play,positive,[other]
1598,e331ee97-7303-40b9-a1ed-3fdd48498cb3,Ayanle Awcali,Good application for people needs,5,2023-03-18,Dashen,Google Play,positive,[other]


In [15]:
# View results
merged_df.groupby('bank')['themes'].value_counts

<bound method SeriesGroupBy.value_counts of <pandas.core.groupby.generic.SeriesGroupBy object at 0x00000188948B5430>>

## Sentiment Analysis:

We use the distilbert-base-uncased-finetuned-sst-2-english model from HuggingFace

This is a distilled version of BERT fine-tuned for sentiment analysis

It classifies text as POSITIVE/NEGATIVE with a confidence score

We process reviews in batches for efficiency